In [ ]:
import spacy
import numpy as np
import pandas as pd
import tqdm
import re
from random import shuffle
import random
random.seed(0)

In [ ]:
df_all_data= pd.read_excel(r"data/ner_training_data.xlsx")

In [ ]:
def get_char_offset(list_words):
    offset = 0
    for wrd in list_words:
        offset += len(wrd)
    offset += len(list_words)
    return offset

# Function to insert row in the dataframe 
def Insert_row_(row_number, df, row_value): 
    # Slice the upper half of the dataframe 
    df1 = df[0:row_number] 
   
    # Store the result of lower half of the dataframe 
    df2 = df[row_number:] 
   
    # Inser the row in the upper half dataframe 
    df1.loc[row_number]=row_value 
   
    # Concat the two dataframes 
    df_result = pd.concat([df1, df2]) 
   
    # Reassign the index labels 
    df_result.index = [*range(df_result.shape[0])] 
   
    # Return the updated dataframe 
    return df_result

import inflect
import math
def get_price_num2word(number):  
    p = inflect.engine()
    return p.number_to_words(int(number))
        
     

In [ ]:
import inflect
def augment_price(df_aug_tag,df_all_data,aug_samples,aug_tag,page_num_start=100000):
    random.seed(0)
    phrases = ['##NUMSTR## dollar',
                'USD (##NUM##)',
                '$ (##NUM## )',
              ]
    p = inflect.engine()
    num_price_range = [10000,20000,30000,40000]
    
    list_df_aug = []
    for idx in range(aug_samples):
        dict_aug_data = {}
        rnd_phrase= random.choice(phrases)
             
        rand_price = random.choice(price)      
        price_word =  get_price_num2word(rand_price)
        
        rnd_phrase=rnd_phrase.replace('##NUM##',str(rand_price))
        rnd_phrase=rnd_phrase.replace('##NUMSTR##',price_word)        

        df_tmp = df_aug_tag.sample()
        df_tmp = df_aug_tag.sample() 
        idx_page = list(df_tmp['page_num'])[0]
        idx_para = list(df_tmp['par_num'])[0]
        start_word = list(df_tmp['start_word'])[0]
        end_word = list(df_tmp['end_word'])[0]
        
        df_tmp_para = df_all_data[(df_all_data['page_num']==idx_page) &  (df_all_data['par_num']==idx_para)]
        df_tmp_para.reset_index(inplace= True)
        print('{}: {} <--> {}'.format(idx,' '.join(list(df_tmp_para.loc[start_word:end_word]['word'])),rnd_phrase))

        df_tmp_para['page_num']= pd.Series([page_num_start]*len(df_tmp_para))
        df_tmp_para['par_num']= pd.Series([idx]*len(df_tmp_para))
        df_tmp_para.drop(list(range(start_word,end_word)),inplace = True) # drop esiting phrase words
        rnd_phrase_words = rnd_phrase.split()
        for idx_wrd,word in enumerate(rnd_phrase_words): # insert new phrase words
            if idx_wrd == 0:
                new_row = [0,page_num_start,idx,word,'B_'+aug_tag,len(rnd_phrase_words)]
            else:
                new_row = [0,page_num_start,idx,word,'I_'+aug_tag,0]
            df_tmp_para = Insert_row_(start_word+idx_wrd, df_tmp_para, new_row)
        df_tmp_para.reset_index(inplace= True)
        list_df_aug.append(df_tmp_para)
    df_augment_all = pd.concat(list_df_aug)
    return df_augment_all.reset_index(drop=True)



In [ ]:

def augment_org(df_aug_tag,df_all_data,aug_samples,aug_tag,page_num_start=600000):
    random.seed(0) 
    phrases = ['abc pvt ltd',
            'xyz pvt ltd',
           
            ]
    list_df_aug = []
    for idx in range(aug_samples):
        dict_aug_data = {}
        rnd_phrase= random.choice(phrases)
        df_tmp = df_aug_tag.sample()
        df_tmp = df_aug_tag.sample() 
        idx_page = list(df_tmp['page_num'])[0]
        idx_para = list(df_tmp['par_num'])[0]
        start_word = list(df_tmp['start_word'])[0]
        end_word = list(df_tmp['end_word'])[0]
        
        df_tmp_para = df_all_data[(df_all_data['page_num']==idx_page) &  (df_all_data['par_num']==idx_para)]
        df_tmp_para.reset_index(inplace= True)
        print('{}: {} <--> {}'.format(idx,' '.join(list(df_tmp_para.loc[start_word:end_word]['word'])),rnd_phrase))

        df_tmp_para['page_num']= pd.Series([page_num_start]*len(df_tmp_para))
        df_tmp_para['par_num']= pd.Series([idx]*len(df_tmp_para))
        df_tmp_para.drop(list(range(start_word,end_word)),inplace = True) # drop esiting phrase words
        rnd_phrase_words = rnd_phrase.split()
        for idx_wrd,word in enumerate(rnd_phrase_words): # insert new phrase words
            if idx_wrd == 0:
                new_row = [0,page_num_start,idx,word,'B_'+aug_tag,len(rnd_phrase_words)]
            else:
                new_row = [0,page_num_start,idx,word,'I_'+aug_tag,0]
            df_tmp_para = Insert_row_(start_word+idx_wrd, df_tmp_para, new_row)
        df_tmp_para.reset_index(inplace= True)
        list_df_aug.append(df_tmp_para)
    df_augment_all = pd.concat(list_df_aug)
    return df_augment_all.reset_index(drop=True)


In [ ]:
augment_tags = ['PRICE','ORG']
aug_samples = 150
# aug_tag = 'Exclusive_Yes'
def get_tag_info_df(aug_tag,df_all_data):
    df_aug_tag = df_all_data[df_all_data['tag']=='B_'+aug_tag]
    df_aug_tag.reset_index(inplace = True)
    list_para = []
    list_start_word = []
    list_end_word = []
    list_start_char = []
    list_end_char = []
    # list_end,idx_tag_end_char
    for index, row in df_aug_tag.iterrows():
        idx_page = row['page_num']
        idx_para = row['par_num']
        df_tmp = df_all_data[(df_all_data['page_num']==idx_page) &  (df_all_data['par_num']==idx_para)]
        df_tmp.reset_index(inplace= True)
        para = ' '.join(list(df_tmp['word']))
        idx_tag_start = df_tmp[df_tmp['tag'] == 'B_'+aug_tag].index[0]
        idx_tag_end = idx_tag_start + df_tmp.iloc[idx_tag_start]['tag_words']
        list_start =list(df_tmp[0:idx_tag_start]['word']) 
        list_end =list(df_tmp[0:idx_tag_end]['word']) 
        idx_tag_start_char = get_char_offset(list_start)
        idx_tag_end_char = get_char_offset(list_end)-1
        list_end_word.append(idx_tag_end)
        list_start_word.append(idx_tag_start)
        list_para.append(para)
        list_start_char.append(idx_tag_start_char)
        list_end_char.append(idx_tag_end_char)

    df_aug_tag['para'] = pd.Series(list_para)
    df_aug_tag['start_word']= pd.Series(list_start_word)
    df_aug_tag['end_word']= pd.Series(list_end_word)
    df_aug_tag['start_char'] = pd.Series(list_start_char)
    df_aug_tag['end_char'] = pd.Series(list_end_char)
    #     offset_tag = df_all_data
    #     start = 
    # len(list_para)


    df_aug_tag.drop(['index'], axis=1,inplace=True)

    return df_aug_tag

In [ ]:
df_aug_tag = get_tag_info_df(aug_tag= 'PRICE',df_all_data=df_all_data)
df_augment_PRICE = augment_price(df_aug_tag,df_all_data,aug_samples=150,aug_tag= 'PRICE')


In [ ]:
df_aug_tag = get_tag_info_df(aug_tag= 'ORG',df_all_data=df_all_data)
df_augment_ORG = augment_org(df_aug_tag,df_all_data,aug_samples=1000,aug_tag= 'ORG')


In [ ]:
df_augment_all = pd.concat([df_augment_PRICE,df_augment_ORG])
df_augment_all.drop(['index'], axis=1,inplace=True)
df_augment_all.reset_index(drop=True,inplace =True)
df_augment_all.to_excel("data/augmented_data.xlsx",index= False)

In [ ]:
df_augment_all.drop(['level_0'], axis=1,inplace=True)
len(df_augment_all)